In [1]:
import os
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Concatenate, Dropout, Flatten, Dense, Conv2DTranspose
from sklearn.model_selection import train_test_split

base_dir = 'F:/Nalaik_Semester'
classes = ['cats', 'dogs']

# Define functions
def load_and_split_data(base_dir, validation_split=0.2):
    all_filenames = []
    labels = []
    for label, class_name in enumerate(classes):
        class_dir = os.path.join(base_dir, class_name)
        for file in os.listdir(class_dir):
            if file.endswith('.jpg') or file.endswith('.png'):
                all_filenames.append(os.path.join(class_dir, file))
                labels.append(label)
    return train_test_split(all_filenames, labels, test_size=validation_split, stratify=labels, random_state=42)

def preprocess_image(filename, label):
    image_string = tf.io.read_file(filename)
    image_decoded = tf.image.decode_jpeg(image_string, channels=3)
    image_resized = tf.image.resize(image_decoded, [144, 144])
    image_normalized = image_resized / 255.0
    return image_normalized, label

def unet_model(input_size=(144, 144, 3)):
    inputs = Input(input_size)
    conv1 = Conv2D(32, 3, activation='relu', padding='same')(inputs)
    conv1 = Conv2D(32, 3, activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    
    conv2 = Conv2D(64, 3, activation='relu', padding='same')(pool1)
    conv2 = Conv2D(64, 3, activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    
    conv3 = Conv2D(128, 3, activation='relu', padding='same')(pool2)
    conv3 = Conv2D(128, 3, activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    
    conv4 = Conv2D(256, 3, activation='relu', padding='same')(pool3)
    conv4 = Conv2D(256, 3, activation='relu', padding='same')(conv4)
    drop4 = Dropout(0.5)(conv4)

    up7 = Conv2DTranspose(128, 3, strides=(2, 2), padding='same')(drop4)
    merge7 = Concatenate()([conv3, up7])
    conv7 = Conv2D(128, 3, activation='relu', padding='same')(merge7)
    
    up8 = Conv2DTranspose(64, 3, strides=(2, 2), padding='same')(conv7)
    merge8 = Concatenate()([conv2, up8])
    conv8 = Conv2D(64, 3, activation='relu', padding='same')(merge8)
    
    up9 = Conv2DTranspose(32, 3, strides=(2, 2), padding='same')(conv8)
    merge9 = Concatenate()([conv1, up9])
    conv9 = Conv2D(32, 3, activation='relu', padding='same')(merge9)
    
    conv10 = Conv2D(1, 1, activation='sigmoid')(conv9)
    
    flatten = Flatten()(conv10)
    dense = Dense(1, activation='sigmoid')(flatten)

    model = Model(inputs=inputs, outputs=dense)
    return model

# Load and preprocess data
filenames_train, filenames_val, y_train, y_val = load_and_split_data(base_dir)
train_dataset = tf.data.Dataset.from_tensor_slices((filenames_train, y_train))
train_dataset = train_dataset.map(preprocess_image).shuffle(len(filenames_train)).batch(32)
val_dataset = tf.data.Dataset.from_tensor_slices((filenames_val, y_val))
val_dataset = val_dataset.map(preprocess_image).batch(32)

# Build and compile model
model = unet_model()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_dataset, epochs=10, validation_data=val_dataset)

# Save the trained model
model.save('trained_model.h5')


Epoch 1/10
16/16 [==============================] - 137s 7s/step - loss: 1.1020 - accuracy: 0.5177 - val_loss: 0.6931 - val_accuracy: 0.5041
Epoch 2/10
16/16 [==============================] - 107s 7s/step - loss: 0.6932 - accuracy: 0.4990 - val_loss: 0.6931 - val_accuracy: 0.5041
Epoch 3/10
16/16 [==============================] - 108s 7s/step - loss: 0.6932 - accuracy: 0.4990 - val_loss: 0.6931 - val_accuracy: 0.5041
Epoch 4/10
16/16 [==============================] - 116s 7s/step - loss: 0.6932 - accuracy: 0.4990 - val_loss: 0.6931 - val_accuracy: 0.5041
Epoch 5/10
16/16 [==============================] - 141s 9s/step - loss: 0.6932 - accuracy: 0.4990 - val_loss: 0.6931 - val_accuracy: 0.5041
Epoch 6/10
16/16 [==============================] - 121s 7s/step - loss: 0.6932 - accuracy: 0.4990 - val_loss: 0.6931 - val_accuracy: 0.5041
Epoch 7/10
16/16 [==============================] - 104s 6s/step - loss: 0.6932 - accuracy: 0.4990 - val_loss: 0.6931 - val_accuracy: 0.5041
Epoch 8/10
16

f:\python\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
